<!-- # SPOTIFY
- ENTER a song Title, Artist Name, Album Name
- Play songs by clicking TRACK ID (url)

<b>NOTE:</b> Requires Spotify Login to play songs -->

In [3]:
from IPython.display import Image
from IPython.core.display import HTML

html1= '<center><img src="https://wallpaperaccess.com/full/1373261.png" width= "650" height="350" ></center>'
#Image(url='https://aldianews.com/sites/default/files/articles/spotify-logo-1920x1080_fouoik-1.jpg', width= 300, height= 200)

HTML(html1)

In [2]:
import datetime
import requests
import numpy as np
import pandas as pd
import base64 # For Encoding

import json
from urllib.parse import urlencode # TO pass URL Code: DATA and Query

# WIDGET
from ipywidgets import widget
from IPython.display import display, clear_output

import ipywidgets as widget
#import warnings

In [3]:
# Client Credentials
client_id= '1cb077b9fdd144fca661a644ea95420b'
client_secret= 'c9485fcc9a7e4f9a900413d7c89b63e2'

In [4]:
# CLASS OBJECT to authenticate KEY 
class SpotifyAPI(object):
    
    # assiging each Value after Retrieving the data
    access_token= None
    access_token_expires= datetime.datetime.now() # current DateTime
    access_token_DID_epxire= True  # BY default
    client_id= None
    client_secret= None
    
    # URL for SPOTIFY api
    token_url= "https://accounts.spotify.com/api/token"
    
    
    # creating a FUNCTION that will call itself for Initialization: 
    # Initializing: Client ID and Client Secret
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # initializing the CLient ID and Client Secret key
        self.client_id= client_id
        self.client_secret= client_secret
    
    # Authentication Purpose:
    # creates base64 credentials by encoding base64 and returns the DECODED version
    def get_client_credentials(self):
        # Update the credentials and CHECK for Verification
        
        client_id= self.client_id
        client_secret= self.client_secret
        
        # To verify Client Credential is EMPTY
        if client_id == None or client_secret ==None:
            raise Exception("You must set client_id and client_secret")
        
        # IF verification PASSED: ENCODE with base64 using F-strp0ing
        client_credentials= f"{client_id}:{client_secret}"
        client_creds_base64= base64.b64encode(client_credentials.encode()) #Encode Creds
        
        return client_creds_base64.decode() #Return the Decoded Creds

    
    def get_token_headers(self):
        # get the Client encoded Credential
        client_cred_b64= self.get_client_credentials()
        
        return{
            "Authorization": f"Basic {client_cred_b64}"
        }
    
    # Get Token Dat
    def get_token_data(self):
        return{
            "grant_type": "client_credentials"
        }
    
    # EXTRACT DATA to Check Authentication using passed TOKEN header, and Client Creds
    
    def perform_authentication(self):
        token_url = self.token_url
        token_data= self.get_token_data()
        token_headers= self.get_token_headers()
        
        # Retrieve Data from the passed URL
        req_data= requests.post(token_url, data= token_data, headers= token_headers) 
        # check STATUS code 
        if req_data.status_code not in range(200, 299):
            return False
        
        # Run this if STATUS code is OK
        data= req_data.json()
        now= datetime.datetime.now() # Get the current Date and Time
        access_token= data['access_token'] # get the AccessTOKEN
        expires_in= data['expires_in'] # Expiry time in seconds
        
        expires= now+ datetime.timedelta(seconds= expires_in)
        self.access_token= access_token # initialziing
        self.access_token_expires= expires
        self.access_token_DID_epxire= expires < now # which will be TRUE always
            
        return True

In [5]:
# AUTHENTICATION with client ID and Client Secret KEY

client_spotify= SpotifyAPI(client_id, client_secret)
client_spotify.perform_authentication()

access_token= search= client_spotify.access_token

headers= {
    "Authorization": f"Bearer {access_token}"
}

# Spotify API for SEARCH endpoint
endpoint= "https://api.spotify.com/v1/search"


# Function to SEARCH song, artist, album, etc.

def get_track_info(str):
    data= urlencode({"q": str, "type":"track"}) # using the 'q' to perform QUERY to get TIME and TRACK

    lookup_url= f"{endpoint}?{data}"
    
    r= requests.get(lookup_url, headers= headers)
    return r.json()


In [10]:
# RETIREVE data from spotify Search function

button = widget.Button(description= "Search")
outPut= widget.Output()

# Search Criteria
srch_title= widget.Text(
    value= '',
    placeholder= 'Enter Song Title, Artist Name'
)

# DISPLAY: display the buttons and search box
display(srch_title)
display(button, outPut)

# WIDGET function apply
# FUNCTION when button CLICKED

def on_button_action(b):
    with outPut:
        # clear output AFTER each SEARCH
        clear_output()
        
        # RETIREVE data from spotify Search function
        track_data= get_track_info(srch_title.value)

        # FUNCTION That returns DataFrame
        def searched_items(meta_data):
            # DataFrame
            # empty lists for the META DATA from JSON format
            Image= []
            Songs= []
            Album= []
            Release_date= []
            Artist= []
            Track_id= []
            #Market= []
            
            # adding HTTP to make a ULR link searchable on SPOTIFY web by ID
            # link_url= 'https://open.spotify.com/track/'

            for i in range(0, len(meta_data['tracks']['items'])):
                
                Image.append(meta_data['tracks']['items'][i]['album']['images'][1].get('url'))
                Songs.append(meta_data['tracks']['items'][i]['name'])
                Album.append(meta_data['tracks']['items'][i]['album']['name'])
                Release_date.append(meta_data['tracks']['items'][i]['album']['release_date'])
                Artist.append(meta_data['tracks']['items'][i]['album']['artists'][0]['name'])
                Track_id.append(meta_data['tracks']['items'][i]['external_urls'].get('spotify'))
                #Market.append(meta_data['tracks']['items'][i]['album']['available_markets'])

            # New dataframe
            #df_data_spot= pd.DataFrame(columns=['Image','Artist','Song', 'Album', 'Release_Date','Track_ID', 'Market'])
            
            
            # without MARKET available info
            df_data_spot= pd.DataFrame(columns=['Image','Artist','Song', 'Album', 'Release_Date','Track_ID'])
            
            df_data_spot['Image']= Image
            df_data_spot['Artist']= Artist
            df_data_spot['Song']= Songs
            df_data_spot['Album']= Album
            df_data_spot['Release_Date']= Release_date
            df_data_spot['Track_ID']= Track_id
            #df_data_spot['Market']= Market

            
            # Function using URL link by SONG ID 
            def make_click(v):
                return '<a href="{}">{}</a>'.format(v, v)

            # Applying the function Create COLUMN of hyperlink (TRCK ID)
            #df_data_spot= df_data_spot.style.format({'Track_ID': make_click})
            
            #df_data_spot.to
            
            return df_data_spot


        
        # LINK to make clickable on Pandas dataframe List
        # Function using URL link by SONG ID 
        def make_click(v):
            return '<a href="{}">{}</a>'.format(v, v)
        
        
        
        # IMAGE cover from URL
        # pass this function for IMAGE column which will have URLs to web
        def path_to_image(path):
            return '<img src="'+ path + '", width= "300">'
        
        # Create NEW dataFrame
        #df_spot_link= searched_items(track_data).style.format({'Track_ID': make_click})
       
  
        #HTML('<style>{}</style>'.format(CSS))
        
        # using display FUNCTION to display the DataFRAME and HTML contents(image)
        #display(searched_items(track_data).iloc[0])
        display(HTML(searched_items(track_data).to_html(escape= False, formatters=dict(Image= path_to_image))))
            
# Click ACTIOn
button.on_click(on_button_action)


Text(value='', placeholder='Enter Song Title, Artist Name')

Button(description='Search', style=ButtonStyle())

Output()

In [7]:
# LIST out the libraries used
# import watermark

In [8]:
# %load_ext watermark
# %watermark -m -p pandas,numpy,json,ipywidgets,requests,IPython